In [1]:
import lmfit
import numpy as np
import copy
import matplotlib.pyplot as plt

from fitting_functions import paramagnon, power_decay_convolved
from matplotlib.ticker import AutoMinorLocator

%matplotlib widget

In [2]:
E_, I_ = np.loadtxt('LSCO_30_LH_grazout.txt', unpack=True, skiprows=1)
E_ *= -1
choose = np.logical_and(E_>-.5, E_<2.5)
E = E_[choose]
I = I_[choose]

dd_onset = 1

In [3]:
model = (lmfit.models.GaussianModel(prefix='el_') 
         + lmfit.models.GaussianModel(prefix='ph_') 
         + lmfit.Model(power_decay_convolved, prefix='cont_')
         + lmfit.models.PseudoVoigtModel(prefix='dd0_')
         + lmfit.models.PseudoVoigtModel(prefix='dd1_')
         + lmfit.models.PseudoVoigtModel(prefix='dd2_')
         + lmfit.models.ConstantModel())
params = model.make_params()

fwhm = 2*np.sqrt(2*np.log(2))
res = 0.13/fwhm
                    
params['el_center'].set(value=0, vary=False)
params['el_amplitude'].set(value=14, min=0)
params['el_sigma'].set(value=res, vary=False)

params['ph_center'].set(value=.09, min=0.05, max=.1, vary=False)
params['ph_sigma'].set(value=res, vary=False, min=0)
params['ph_amplitude'].set(value=10, min=0) #!!!!!!!!

params['cont_center'].set(value=.23, min=0, max=1)  # expr='mag_center'
params['cont_eta'].set(value=.5, min=0.01, max=1.)
params['cont_amplitude'].set(value=20, min=0)
params['cont_res'].set(value=res*1.5, min=0)


params['dd0_center'].set(value=1.6, min=1, max=3)
params['dd0_sigma'].set(value=0.1, min=0)
params['dd0_amplitude'].set(value=300)

params['dd1_center'].set(value=1.8, min=1, max=3)
params['dd1_sigma'].set(value=0.1, min=0)
params['dd1_amplitude'].set(value=300)

params['dd2_center'].set(value=2, min=1, max=3)
params['dd2_sigma'].set(value=0.1, min=0)
params['dd2_amplitude'].set(value=300)

params_dd = copy.deepcopy(params)

for key in params_dd.keys():
    if key[:2] in ['el', 'ph', 'cont']:
        params_dd[key].set(vary=False)
    else:
        params_dd[key].set(vary=True)

# Fit dds and force leading edge accuracy  by artificial weighting
dd_region = np.logical_or(E<-.3, E>dd_onset)
weights = .2 + np.exp(-1*((E-dd_onset)/.5)**2)
params_dd['c'].set(value=I.min(), vary=False)       
result_dds = model.fit(I[dd_region], x=E[dd_region], params=params_dd,
                      weights=weights[dd_region])

# fig, ax = plt.subplots()
# result_dds.plot_fit(ax=ax, show_init=True)

# assign values for dds 
for key in params.keys():
    if key[:2] == 'dd':
        params[key].set(value=result_dds.params[key].value, vary=True)

params['c'].set(value=I.min(), vary=False)       
result = model.fit(I, x=E, params=params)

fig, ax = plt.subplots()
result.plot_fit(ax=ax, show_init=True)
 
#print(result.fit_report())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
fig, ax = plt.subplots()

x_fit = np.linspace(E.min(), E.max(), 10000)

components = result.eval_components(x=x_fit)
y_fit = result.eval(x=x_fit)

constant = components.pop('constant')
dd0 = components.pop('dd0_')
dd1 = components.pop('dd1_')
dd2 = components.pop('dd2_')

BG = constant + dd0 + dd1 + dd2

ax.plot(x_fit, BG, 'k:', label='BG')
for model_name, model_value in components.items():
    ax.plot(x_fit, model_value + BG, '-', label=model_name.strip('_'))

ax.plot(x_fit, y_fit, color=[0.5]*3, label='fit', lw=3, alpha=0.5)
ax.plot(E, I, 'k.', label='data')

ax.set_xlabel('Energy loss (eV)')
ax.set_ylabel('I')
ax.legend()
ax.axis([-.4, dd_onset, 0, 400])

ax.xaxis.set_minor_locator(AutoMinorLocator(2))
ax.yaxis.set_minor_locator(AutoMinorLocator(2))

print(result.fit_report())

result.dump(open('fit_info.json','w'))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[Model]]
    ((((((Model(gaussian, prefix='el_') + Model(gaussian, prefix='ph_')) + Model(power_decay_convolved, prefix='cont_')) + Model(pvoigt, prefix='dd0_')) + Model(pvoigt, prefix='dd1_')) + Model(pvoigt, prefix='dd2_')) + Model(constant))
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 53
    # data points      = 98
    # variables        = 24
    chi-square         = 24534.6624
    reduced chi-square = 331.549492
    Akaike info crit   = 589.241722
    Bayesian info crit = 651.280941
##  Warning: uncertainties could not be estimated:
    cont_res:        at initial value
    dd0_fwhm:        at initial value
    dd0_height:      at initial value
    dd1_fwhm:        at initial value
    dd1_height:      at initial value
    dd2_fwhm:        at initial value
    dd2_height:      at initial value
[[Variables]]
    el_amplitude:    9.41057924 (init = 14)
    el_center:       0 (fixed)
    el_sigma:        0.05520592 (fixed)
    ph_amplitude:    6.37732

17283